In [21]:
import requests
import pandas as pd

authkey = ''
start_date = '20230508'
end_date = '20230608'
data_type = 'AP01'  # AP01 : 환율, AP02 : 대출금리, AP03 : 국제금리

date_range = pd.date_range(start_date, end_date)

df = pd.DataFrame()

for search_date in date_range:
    search_date_str = search_date.strftime("%Y%m%d")
    url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey={}&searchdate={}&data={}'.format(authkey, search_date_str, data_type)

    response = requests.get(url)
    r_data = response.json()
    exchange_rate_summary = pd.DataFrame(r_data)
    exchange_rate_summary['date'] = search_date
    df = pd.concat([df, exchange_rate_summary], ignore_index=True)


df.to_csv('exchange_rate_data.csv', index=False, encoding='utf-8-sig')
#result             조회 결과
#cur_unit           통화코드
#ttb                전신환(송금)받으실때
#tts                전신환(송금)보내실때
#deal_bas_r         매매 기준율
#bkpr               장부가격
#yy_efee_r          년환가료율
#ten_dd_efee_r      10일환가료율
#kftc_bkpr          서울외국환중개 매매기준율
#kftc_deal_bas_r    서울외국환중개 장부가격
#cur_nm             국가/통화명

In [26]:
exchange_rate_summary = df
exchange_rate_summary['cur_unit'] = exchange_rate_summary['cur_unit'].str.replace(r'\(100\)', '', regex=True)
exchange_rate_summary

,result,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm,date
0,1.0,AED,357.39,364.61,361,361,0,0,361,361,아랍에미리트 디르함,2023-05-08
1,1.0,AUD,886.28,904.19,895.24,895,0,0,895,895.24,호주 달러,2023-05-08
2,1.0,BHD,"3,480.74","3,551.05","3,515.9","3,515",0,0,"3,515","3,515.9",바레인 디나르,2023-05-08
3,1.0,BND,989.81,"1,009.8",999.81,999,0,0,999,999.81,브루나이 달러,2023-05-08
4,1.0,CAD,980.93,"1,000.74",990.84,990,0,0,990,990.84,캐나다 달러,2023-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...
501,1.0,SAR,343.32,350.25,346.79,346,0,0,346,346.79,사우디 리얄,2023-06-08
502,1.0,SEK,118.13,120.52,119.33,119,0,0,119,119.33,스웨덴 크로나,2023-06-08
503,1.0,SGD,954.58,973.87,964.23,964,0,0,964,964.23,싱가포르 달러,2023-06-08
504,1.0,THB,36.94,37.69,37.32,37,0,0,37,37.32,태국 바트,2023-06-08


In [28]:
df.to_csv('exchange_rate_data.csv', index=False, encoding='utf-8-sig')

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

start_date = '2023-05-08'
end_date = '2023-06-08'
date_range = pd.date_range(start_date, end_date)

data = []

for search_date in date_range:
    url = 'https://www.kita.net/cmmrcInfo/ehgtGnrlzInfo/rltmEhgt.do?sDate={}'.format(search_date.strftime("%Y-%m-%d"))
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows:
        currency = row.find('a')
        change = row.find('span')

        if currency and change:
            currency_name = currency.text.strip()
            change_data = change.text.strip()
            # 치환 및 정리
            change_data = change_data.replace('▼', '-').replace('▲', '+').replace('\t', '')
            data.append({'통화명': currency_name, '전일대비': change_data, 'date': search_date})

    time.sleep(2)  # 2초 대기

df = pd.DataFrame(data)
df['전일대비'] = df['전일대비'].str.replace('\n', '')  # '\n' 제거
print(df)


           통화명   전일대비       date
0       USD 미국  +0.60 2023-05-08
1       JPY 일본  -5.14 2023-05-08
2       EUR 유럽  -0.27 2023-05-08
3       CNY 중국  +0.11 2023-05-08
4       HKD 홍콩  +0.09 2023-05-08
..         ...    ...        ...
611  BDT 방글라데시  -0.07 2023-06-08
612    PHP 필리핀  -0.15 2023-06-08
613    EGP 이집트  -0.34 2023-06-08
614    MXN 멕시코  -0.41 2023-06-08
615   BND 브루나이  -6.66 2023-06-08

[616 rows x 3 columns]


In [30]:
df['통화명'] = df['통화명'].str.split().str[0]
print(df)


     통화명   전일대비       date
0    USD  +0.60 2023-05-08
1    JPY  -5.14 2023-05-08
2    EUR  -0.27 2023-05-08
3    CNY  +0.11 2023-05-08
4    HKD  +0.09 2023-05-08
..   ...    ...        ...
611  BDT  -0.07 2023-06-08
612  PHP  -0.15 2023-06-08
613  EGP  -0.34 2023-06-08
614  MXN  -0.41 2023-06-08
615  BND  -6.66 2023-06-08

[616 rows x 3 columns]


In [31]:
df.to_csv('crawling.csv', index=False, encoding='utf-8-sig')

In [32]:
crawling = df
crawling

,통화명,전일대비,date
0,USD,+0.60,2023-05-08
1,JPY,-5.14,2023-05-08
2,EUR,-0.27,2023-05-08
3,CNY,+0.11,2023-05-08
4,HKD,+0.09,2023-05-08
...,...,...,...
611,BDT,-0.07,2023-06-08
612,PHP,-0.15,2023-06-08
613,EGP,-0.34,2023-06-08
614,MXN,-0.41,2023-06-08


In [33]:
# 필요한 컬럼만 선택하여 데이터프레임 생성
exchange_rate_summary_filtered = exchange_rate_summary[['cur_unit', 'date', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r', 'cur_nm']]
df_filtered = df[['통화명', '전일대비', 'date']]

merged_df_list = []

# 날짜 범위 설정
start_date = pd.to_datetime('2023-05-08')
end_date = pd.to_datetime('2023-06-08')
date_range = pd.date_range(start_date, end_date)

# 날짜별로 조인 수행
for date in date_range:
    date_str = date.strftime('%Y%m%d')
    date_filtered = df_filtered[df_filtered['date'] == date_str]
    exchange_rate_filtered = exchange_rate_summary_filtered[exchange_rate_summary_filtered['date'] == date_str]
    
    merged_df = pd.merge(date_filtered, exchange_rate_filtered, left_on=['통화명', 'date'], right_on=['cur_unit', 'date'], how='left')
    merged_df_list.append(merged_df)

# 모든 데이터프레임 병합
merged_df = pd.concat(merged_df_list)

# 필요한 컬럼만 선택
merged_df = merged_df[['통화명', '전일대비', 'date', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r', 'cur_nm']]

# 결과 출력
print(merged_df)


    통화명   전일대비       date       ttb       tts deal_bas_r   bkpr kftc_bkpr   
0   USD  +0.60 2023-05-08  1,312.34  1,338.85    1,325.6  1,325     1,325  \
1   JPY  -5.14 2023-05-08    971.06    990.67     980.87    980       980   
2   EUR  -0.27 2023-05-08   1,445.8  1,475.01   1,460.41  1,460     1,460   
3   CNY  +0.11 2023-05-08       NaN       NaN        NaN    NaN       NaN   
4   HKD  +0.09 2023-05-08    167.23     170.6     168.92    168       168   
..  ...    ...        ...       ...       ...        ...    ...       ...   
23  BDT  -0.07 2023-06-08       NaN       NaN        NaN    NaN       NaN   
24  PHP  -0.15 2023-06-08       NaN       NaN        NaN    NaN       NaN   
25  EGP  -0.34 2023-06-08       NaN       NaN        NaN    NaN       NaN   
26  MXN  -0.41 2023-06-08       NaN       NaN        NaN    NaN       NaN   
27  BND  -6.66 2023-06-08    954.58    973.87     964.23    964       964   

   kftc_deal_bas_r   cur_nm  
0          1,325.6    미국 달러  
1           980

In [29]:
merged_df = pd.merge(df, exchange_rate_summary, left_on='통화명', right_on='cur_unit')
merged_df

,통화명,전일대비,date_x,result,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm,date_y
0,USD,+0.60,2023-05-08,1,USD,"1,287.49","1,313.5","1,300.5","1,300",0,0,"1,300","1,300.5",미국 달러,2023-06-08
1,USD,-0.30,2023-05-09,1,USD,"1,287.49","1,313.5","1,300.5","1,300",0,0,"1,300","1,300.5",미국 달러,2023-06-08
2,USD,-4.00,2023-05-10,1,USD,"1,287.49","1,313.5","1,300.5","1,300",0,0,"1,300","1,300.5",미국 달러,2023-06-08
3,USD,+3.70,2023-05-11,1,USD,"1,287.49","1,313.5","1,300.5","1,300",0,0,"1,300","1,300.5",미국 달러,2023-06-08
4,USD,-12.40,2023-05-12,1,USD,"1,287.49","1,313.5","1,300.5","1,300",0,0,"1,300","1,300.5",미국 달러,2023-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,BND,-2.40,2023-06-01,1,BND,954.58,973.87,964.23,964,0,0,964,964.23,브루나이 달러,2023-06-08
414,BND,+6.74,2023-06-02,1,BND,954.58,973.87,964.23,964,0,0,964,964.23,브루나이 달러,2023-06-08
415,BND,-2.00,2023-06-05,1,BND,954.58,973.87,964.23,964,0,0,964,964.23,브루나이 달러,2023-06-08
416,BND,+4.91,2023-06-07,1,BND,954.58,973.87,964.23,964,0,0,964,964.23,브루나이 달러,2023-06-08


In [36]:
merged_df.dropna(inplace=True)
merged_df

,통화명,전일대비,date,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,USD,+0.60,2023-05-08,"1,312.34","1,338.85","1,325.6","1,325","1,325","1,325.6",미국 달러
1,JPY,-5.14,2023-05-08,971.06,990.67,980.87,980,980,980.87,일본 옌
2,EUR,-0.27,2023-05-08,"1,445.8","1,475.01","1,460.41","1,460","1,460","1,460.41",유로
4,HKD,+0.09,2023-05-08,167.23,170.6,168.92,168,168,168.92,홍콩 달러
5,GBP,+1.75,2023-05-08,"1,657.28","1,690.77","1,674.03","1,674","1,674","1,674.03",영국 파운드
...,...,...,...,...,...,...,...,...,...,...
17,THB,-0.31,2023-06-08,36.94,37.69,37.32,37,37,37.32,태국 바트
18,SGD,-7.20,2023-06-08,954.58,973.87,964.23,964,964,964.23,싱가포르 달러
19,IDR,-0.06,2023-06-08,8.65,8.82,8.74,8,8,8.74,인도네시아 루피아
21,MYR,-1.36,2023-06-08,280.04,285.69,282.87,282,282,282.87,말레이지아 링기트


In [37]:
merged_df.to_csv('merged_data.csv', index=False, encoding='utf-8-sig')